# Mysql 데이터베이스 연동

In [1]:
import pymysql

In [2]:
# 데이터 베이스 접속
conn = pymysql.connect(
    host = 'localhost',         # 서버의 호스트 이름 또는 ip주소
    user = 'root',              # mysql 접속시 id
    password = '0000',          # password
    database = 'movie_db',      # 연결할 데이터베이스
    charset = 'utf8'            # 데이터베이스의 문자 세트
)

In [3]:
# 커서 생성
cursor = conn.cursor()

## 쿼리문 사용

### DROP TABLE - 데이터가 아닌 테이블 삭제!!!!!!!!!!!

In [4]:
# 쿼리문 작성 - DROP 테이블
# 조건. 현재 학습 중이므로 테이블이 존재하면 삭제후 테이블 생성
# 실무에서는 절대 테이블 중요도 보고 삭제
query = 'DROP TABLE if exists week_movie;'

# 커서이용해서 쿼리 실행
cursor.execute(query=query)

# 서버에 현재 트랜잭션을 커밋
conn.commit()

### CREATE TABLE - 테이블 생성

In [5]:
# 쿼리문 작성 - CREATE 테이블
# 데이터 저장할 테이블 생성 <-- 여기에 PK, NOT NULL, foreign key 등 지정
query = 'CREATE TABLE week_movie(showRange VARCHAR(50), ranking INT, rankInten INT, rankOldAndNew VARCHAR(3), movieCd VARCHAR(20) NOT NULL UNIQUE, movieNm VARCHAR(50),openDt DATE,audiCnt INT, audiInten INT, audiAcc INT, scrnCnt INT, showCnt INT);'

# 쿼리 실행
cursor.execute(query=query)

# 서버에 현재 트랜잭션을 커밋
conn.commit()

### INSERT INTO - 테이블에 데이터 넣기
- 사용한 api 사이트 : https://www.kobis.or.kr/kobisopenapi/homepg/apiservice/searchServiceInfo.do

In [6]:
import requests
import json
import movie

In [7]:
# 절대 비공개 해야함.
key = movie.key

# get방식으로 api 호출시 필요한 파라미터 값
params = {
    'targetDt' : '20231212',
    'weekGb' : '0'
}
# url
url_api = f'http://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchWeeklyBoxOfficeList.json?key={key}'

In [8]:
# get 방식으로 전송상태 확인
response = requests.get(url_api, params=params)
response

<Response [200]>

In [9]:
# 나온 결과 json 형태로 받기때문에 데이터 
json_data = response.content.decode(encoding='utf-8')
json_data

'{"boxOfficeResult":{"boxofficeType":"주간 박스오피스","showRange":"20231211~20231217","yearWeekTime":"202350","weeklyBoxOfficeList":[{"rnum":"1","rank":"1","rankInten":"0","rankOldAndNew":"OLD","movieCd":"20212866","movieNm":"서울의 봄","openDt":"2023-11-22","salesAmt":"19298138579","salesShare":"70.8","salesInten":"-3369142460","salesChange":"-14.9","salesAcc":"86500588671","audiCnt":"1968942","audiInten":"-347892","audiChange":"-15.0","audiAcc":"8940820","scrnCnt":"2313","showCnt":"56850"},{"rnum":"2","rank":"2","rankInten":"0","rankOldAndNew":"OLD","movieCd":"20190324","movieNm":"3일의 휴가","openDt":"2023-12-06","salesAmt":"1970797079","salesShare":"7.2","salesInten":"-57998738","salesChange":"-2.9","salesAcc":"4026993896","audiCnt":"209297","audiInten":"-4288","audiChange":"-2.0","audiAcc":"425865","scrnCnt":"861","showCnt":"13992"},{"rnum":"3","rank":"3","rankInten":"4","rankOldAndNew":"OLD","movieCd":"20234673","movieNm":"뽀로로 극장판 슈퍼스타 대모험","openDt":"2023-12-13","salesAmt":"1098821230","salesS

In [10]:
movie_list = json.loads(json_data)
# movie_list

In [11]:
movie_list['boxOfficeResult']['weeklyBoxOfficeList'][0]

{'rnum': '1',
 'rank': '1',
 'rankInten': '0',
 'rankOldAndNew': 'OLD',
 'movieCd': '20212866',
 'movieNm': '서울의 봄',
 'openDt': '2023-11-22',
 'salesAmt': '19298138579',
 'salesShare': '70.8',
 'salesInten': '-3369142460',
 'salesChange': '-14.9',
 'salesAcc': '86500588671',
 'audiCnt': '1968942',
 'audiInten': '-347892',
 'audiChange': '-15.0',
 'audiAcc': '8940820',
 'scrnCnt': '2313',
 'showCnt': '56850'}

In [12]:
# 데이터 총 개수 확인
len(movie_list['boxOfficeResult']['weeklyBoxOfficeList'])

10

In [13]:
# 위에서 얻은 데이터로 INSERT INTO 바로 실행
version = 1

showRange = movie_list['boxOfficeResult']['showRange']

for _ in range(0, len(movie_list['boxOfficeResult']['weeklyBoxOfficeList'])):
    ranking = movie_list['boxOfficeResult']['weeklyBoxOfficeList'][_]['rank']
    rankInten = movie_list['boxOfficeResult']['weeklyBoxOfficeList'][_]['rankInten']
    rankOldAndNew = movie_list['boxOfficeResult']['weeklyBoxOfficeList'][_]['rankOldAndNew']
    movieCd = movie_list['boxOfficeResult']['weeklyBoxOfficeList'][_]['movieCd']
    movieNm = movie_list['boxOfficeResult']['weeklyBoxOfficeList'][_]['movieNm']
    openDt = movie_list['boxOfficeResult']['weeklyBoxOfficeList'][_]['openDt']
    audiCnt = movie_list['boxOfficeResult']['weeklyBoxOfficeList'][_]['audiCnt']
    audiInten = movie_list['boxOfficeResult']['weeklyBoxOfficeList'][_]['audiInten']
    audiAcc = movie_list['boxOfficeResult']['weeklyBoxOfficeList'][_]['audiAcc']
    scrnCnt = movie_list['boxOfficeResult']['weeklyBoxOfficeList'][_]['scrnCnt']
    showCnt = movie_list['boxOfficeResult']['weeklyBoxOfficeList'][_]['showCnt']

    if version == 1:
        # 버전1. INSERT INTO 테이블 컬럼지정X VALUES(컬럼 순서대로 데이터)
        data = [showRange, ranking, rankInten, rankOldAndNew, movieCd, movieNm, openDt, audiCnt, audiInten, audiAcc, scrnCnt, showCnt]
        query = f'INSERT INTO week_movie VALUES ({data});'.replace('[', '').replace(']', '')
        
        # 쿼리 실행
        cursor.execute(query=query)
        print(query, '저장 완료')
        # 서버에 현재 트랜잭션을 커밋
        conn.commit()

    elif version == 2:
        # 버전2. INSERT INTO 테이블 (컬럼지정) VALUES(컬럼 == 데이터)
        data = [showRange, ranking, rankInten, rankOldAndNew, movieCd, movieNm, openDt, audiCnt, audiInten, audiAcc, scrnCnt, showCnt]
        query = f'INSERT INTO week_movie (showRange, ranking, rankInten, rankOldAndNew, movieCd, movieNm, openDt, audiCnt, audiInten, audiAcc, scrnCnt, showCnt) VALUES ({data});'.replace('[', '').replace(']', '')

        # 쿼리 실행
        cursor.execute(query=query)
        print(query, '저장 완료')

        # 서버에 현재 트랜잭션을 커밋
        conn.commit()


INSERT INTO week_movie VALUES ('20231211~20231217', '1', '0', 'OLD', '20212866', '서울의 봄', '2023-11-22', '1968942', '-347892', '8940820', '2313', '56850'); 저장 완료
INSERT INTO week_movie VALUES ('20231211~20231217', '2', '0', 'OLD', '20190324', '3일의 휴가', '2023-12-06', '209297', '-4288', '425865', '861', '13992'); 저장 완료
INSERT INTO week_movie VALUES ('20231211~20231217', '3', '4', 'OLD', '20234673', '뽀로로 극장판 슈퍼스타 대모험', '2023-12-13', '123434', '87751', '159693', '850', '5414'); 저장 완료
INSERT INTO week_movie VALUES ('20231211~20231217', '4', '1', 'OLD', '20234114', '괴물', '2023-11-29', '84258', '-2286', '288601', '433', '4155'); 저장 완료
INSERT INTO week_movie VALUES ('20231211~20231217', '5', '-2', 'OLD', '20235098', '나폴레옹', '2023-12-06', '68524', '-65865', '208100', '731', '6839'); 저장 완료
INSERT INTO week_movie VALUES ('20231211~20231217', '6', '29', 'OLD', '20235698', '쏘우 X', '2023-12-13', '48635', '47010', '50318', '571', '5251'); 저장 완료
INSERT INTO week_movie VALUES ('20231211~20231217', '7', 

### UPDATE TABLE - 데이터 수정

In [14]:
query = 'UPDATE week_movie SET rankOldAndNew="신규" WHERE rankOldAndNew="NEW";'

cursor.execute(query=query)
print(query, '수정 완료')

# 서버에 현재 트랜잭션을 커밋
conn.commit()

UPDATE week_movie SET rankOldAndNew="신규" WHERE rankOldAndNew="NEW"; 수정 완료


### DELETE FROM 테이블 - 테이블의 행 삭제 OR 테이블 데이터 전체삭제

In [15]:
version = 1

if version == 1:
    # 테이블의 행삭제
    query = 'DELETE FROM week_movie WHERE rankOldAndNew="신규";'

    cursor.execute(query=query)
    print(query, '데이터 삭제')

    # 서버에 현재 트랜잭션을 커밋
    conn.commit()
    
elif version == 2:
    # 테이블의 전체 데이터 삭제
    query = 'DELETE FROM week_movie;'

    cursor.execute(query=query)
    print(query, '테이블 전체 데이터 삭제')

    # 서버에 현재 트랜잭션을 커밋
    conn.commit()


DELETE FROM week_movie WHERE rankOldAndNew="신규"; 데이터 삭제


### SELECT문 - 테이블 조회

In [19]:
query = 'SELECT * FROM week_movie;'

cursor.execute(query=query)

# fetchall() : 지정 테이블 안의 모든 데이터 추출
data = cursor.fetchall()

for _ in data:
    print(_)

('20231211~20231217', 2, 0, 'OLD', '20190324', '3일의 휴가', datetime.date(2023, 12, 6), 209297, -4288, 425865, 861, 13992)
('20231211~20231217', 7, -3, 'OLD', '20202247', '싱글 인 서울', datetime.date(2023, 11, 29), 46107, -63528, 381205, 646, 5012)
('20231211~20231217', 1, 0, 'OLD', '20212866', '서울의 봄', datetime.date(2023, 11, 22), 1968942, -347892, 8940820, 2313, 56850)
('20231211~20231217', 4, 1, 'OLD', '20234114', '괴물', datetime.date(2023, 11, 29), 84258, -2286, 288601, 433, 4155)
('20231211~20231217', 3, 4, 'OLD', '20234673', '뽀로로 극장판 슈퍼스타 대모험', datetime.date(2023, 12, 13), 123434, 87751, 159693, 850, 5414)
('20231211~20231217', 10, -4, 'OLD', '20235030', '프레디의 피자가게', datetime.date(2023, 11, 15), 19251, -19172, 709645, 230, 1286)
('20231211~20231217', 5, -2, 'OLD', '20235098', '나폴레옹', datetime.date(2023, 12, 6), 68524, -65865, 208100, 731, 6839)
('20231211~20231217', 6, 29, 'OLD', '20235698', '쏘우 X', datetime.date(2023, 12, 13), 48635, 47010, 50318, 571, 5251)
('20231211~20231217', 8, 23,